In [1]:
from pathlib import Path
import numpy as np

processed_dir = Path("../data_processed")

X      = np.load(processed_dir / "X.npy")
labels = np.load(processed_dir / "labels.npy")

print("X shape:", X.shape)
print("labels shape:", labels.shape)

X shape: (5760, 69)
labels shape: (5760,)


In [2]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    X,
    labels,
    test_size=0.2,
    random_state=42,
    stratify=labels
)

In [3]:
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score, roc_auc_score

# 1. Logistic Regression
lr = LogisticRegression(max_iter=1000)
lr.fit(X_train, y_train)
lr_pred = lr.predict(X_test)
lr_prob = lr.predict_proba(X_test)[:, 1]

print("Logistic Regression")
print("Accuracy:", accuracy_score(y_test, lr_pred))
print("ROC-AUC:", roc_auc_score(y_test, lr_prob))
print()

# 2. SVM (with probability outputs)
svm = SVC(kernel="rbf", probability=True)
svm.fit(X_train, y_train)
svm_pred = svm.predict(X_test)
svm_prob = svm.predict_proba(X_test)[:, 1]

print("SVM")
print("Accuracy:", accuracy_score(y_test, svm_pred))
print("ROC-AUC:", roc_auc_score(y_test, svm_prob))
print()

# 3. MLP
mlp = MLPClassifier(hidden_layer_sizes=(64, 32), max_iter=300, random_state=42)
mlp.fit(X_train, y_train)
mlp_pred = mlp.predict(X_test)
mlp_prob = mlp.predict_proba(X_test)[:, 1]

print("MLP")
print("Accuracy:", accuracy_score(y_test, mlp_pred))
print("ROC-AUC:", roc_auc_score(y_test, mlp_prob))

Logistic Regression
Accuracy: 0.8862847222222222
ROC-AUC: 0.42673325806909856

SVM
Accuracy: 0.8897569444444444
ROC-AUC: 0.799754768188649

MLP
Accuracy: 0.8862847222222222
ROC-AUC: 0.5765826049898692


In [4]:
results_probabilities = {
    "LR":  lr_prob,   # logistic regression probabilities
    "SVM": svm_prob,  # SVM probabilities
    "MLP": mlp_prob   # neural network probabilities
}

print("Saved model probabilities!")

Saved model probabilities!


In [5]:
for model in ["MLP"]:
    prob = results_probabilities[model]
    ss, tiw, bs, bss = compute_metrics(y_test, prob)
    print(f"\nForecasting Metrics for {model}:")
    print("SS:", ss)
    print("TIW:", tiw)
    print("Brier Score:", bs)
    print("Brier Skill Score:", bss)

NameError: name 'compute_metrics' is not defined

In [6]:
from pathlib import Path
import numpy as np

processed_dir = Path("../data_processed")

# Get all feature files for chb01
feature_files = sorted(f for f in processed_dir.glob("chb01_*_features.npy"))

X_list = []
y_list = []

for f in feature_files:
    # load features
    X_f = np.load(f)
    # load corresponding labels file
    labels_f = np.load(processed_dir / f.name.replace("_features", "_labels"))
    
    X_list.append(X_f)
    y_list.append(labels_f)

# Stack everything into one big matrix
X = np.vstack(X_list)
labels = np.concatenate(y_list)

print("X shape:", X.shape)        # should be (5760, 69)
print("labels shape:", labels.shape)  # (5760,)

X shape: (5760, 69)
labels shape: (5760,)


In [7]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, roc_auc_score
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier

# Train / test split
X_train, X_test, y_train, y_test = train_test_split(
    X, labels, test_size=0.2, random_state=42, stratify=labels
)

# ---------- Logistic Regression ----------
lr_clf = LogisticRegression(max_iter=1000)
lr_clf.fit(X_train, y_train)
lr_pred = lr_clf.predict(X_test)
lr_prob = lr_clf.predict_proba(X_test)[:, 1]

print("LR Accuracy:", accuracy_score(y_test, lr_pred))
print("LR ROC-AUC:", roc_auc_score(y_test, lr_prob))

# ---------- SVM ----------
svm_clf = SVC(kernel="rbf", probability=True)
svm_clf.fit(X_train, y_train)
svm_pred = svm_clf.predict(X_test)
svm_prob = svm_clf.predict_proba(X_test)[:, 1]

print("\nSVM Accuracy:", accuracy_score(y_test, svm_pred))
print("SVM ROC-AUC:", roc_auc_score(y_test, svm_prob))

# ---------- MLP ----------
mlp_clf = MLPClassifier(hidden_layer_sizes=(64, 32), max_iter=200)
mlp_clf.fit(X_train, y_train)
mlp_pred = mlp_clf.predict(X_test)
mlp_prob = mlp_clf.predict_proba(X_test)[:, 1]

print("\nMLP Accuracy:", accuracy_score(y_test, mlp_pred))
print("MLP ROC-AUC:", roc_auc_score(y_test, mlp_prob))

LR Accuracy: 0.8862847222222222
LR ROC-AUC: 0.42673325806909856

SVM Accuracy: 0.8897569444444444
SVM ROC-AUC: 0.799754768188649

MLP Accuracy: 0.8862847222222222
MLP ROC-AUC: 0.6384027035311886


In [8]:
def compute_metrics(y_true, prob, window_size_sec=5, threshold=0.7):
    """
    y_true : 0/1 labels per window
    prob   : predicted probabilities per window
    window_size_sec : length of each window (5 s here)
    threshold : firing power threshold -> warning state
    
    Returns:
        SS   : Seizure Sensitivity (TPR)
        FPRh : False Positives per Hour
        TiW  : Time in Warning (fraction of time in alarm)
        BS   : Brier Score
        BSS  : Brier Skill Score
    """
    import numpy as np
    
    y_true = np.asarray(y_true)
    prob = np.asarray(prob)

    # Warning state based on probability threshold
    warn = (prob >= threshold).astype(int)

    # --- SS (sensitivity = TPR on seizure windows) ---
    tp = np.sum((warn == 1) & (y_true == 1))
    fn = np.sum((warn == 0) & (y_true == 1))
    ss = tp / (tp + fn + 1e-9)

    # --- FPR per hour ---
    fp = np.sum((warn == 1) & (y_true == 0))
    hours = len(y_true) * window_size_sec / 3600.0
    fpr_h = fp / (hours + 1e-9)

    # --- Time in Warning (fraction of windows in alarm) ---
    tiw = warn.mean()

    # --- Brier score ---
    bs = np.mean((prob - y_true) ** 2)

    # --- Brier Skill Score (vs climatology) ---
    p = y_true.mean()
    bs_ref = np.mean((p - y_true) ** 2)
    bss = 1.0 - bs / (bs_ref + 1e-9)

    return ss, fpr_h, tiw, bs, bss

In [9]:
results_probabilities = {
    "LR": lr_prob,   # logistic regression probabilities
    "SVM": svm_prob, # SVM probabilities
    "MLP": mlp_prob  # neural network probabilities
}

print("Saved model probabilities!")

Saved model probabilities!


In [10]:
for model_name in ["LR", "SVM", "MLP"]:
    prob = results_probabilities[model_name]
    ss, fpr_h, tiw, bs, bss = compute_metrics(y_test, prob)

    print(f"\nForecasting Metrics for {model_name}:")
    print("SS      :", ss)
    print("FPR/h   :", fpr_h)
    print("TiW     :", tiw)
    print("Brier   :", bs)
    print("Brier SS:", bss)


Forecasting Metrics for LR:
SS      : 0.0
FPR/h   : 0.0
TiW     : 0.0
Brier   : 0.10078419571713916
Brier SS: -8.070660646453121e-07

Forecasting Metrics for SVM:
SS      : 0.09160305343441526
FPR/h   : 0.6249999996093749
TiW     : 0.011284722222222222
Brier   : 0.08177510880687788
Brier SS: 0.1886111287299126

Forecasting Metrics for MLP:
SS      : 0.0
FPR/h   : 0.0
TiW     : 0.0
Brier   : 0.10078383412590834
Brier SS: 2.780713944772195e-06
